In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import models as torchvision_models
from torchvision.models import EfficientNet_B0_Weights
from torchvision import transforms
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
from collections import Counter

## Diccionario para Piezas del Vehículo (completo y corregido)
label_to_cls_piezas = {
    1: "Antiniebla delantero derecho",
    2: "Antiniebla delantero izquierdo",
    3: "Asiento",
    4: "Brazo del techo",
    5: "Brazo transversal",
    6: "Capó",
    7: "Cerradura capo",
    8: "Cerradura maletero",
    9: "Cerradura puerta",
    10: "Espejo lateral derecho",
    11: "Espejo lateral izquierdo",
    12: "Espejo retrovisor",
    13: "Faros derecho",
    14: "Faros izquierdo",
    15: "Guardabarros delantero derecho",
    16: "Guardabarros delantero izquierdo",
    17: "Guardabarros trasero derecho",
    18: "Guardabarros trasero izquierdo",
    19: "Limpiaparabrisas",
    20: "Luz indicadora delantera derecha",
    21: "Luz indicadora delantera izquierda",
    22: "Luz indicadora trasera derecha",
    23: "Luz indicadora trasera izquierda",
    24: "Luz trasera derecho",
    25: "Luz trasera izquierdo",
    26: "Maletero",
    27: "Manija derecha",
    28: "Manija izquierda",
    29: "Marco de la ventana",
    30: "Marco de las puertas",
    31: "Matrícula",
    32: "Moldura capó",
    33: "Moldura maletro",
    34: "Moldura puerta delantera derecha",
    35: "Moldura puerta delantera izquierda",
    36: "Moldura puerta trasera derecha",
    37: "Moldura puerta trasera izquierda",
    38: "Parabrisas delantero",
    39: "Parabrisas trasero",
    40: "Parachoques delantero",
    41: "Parachoques trasero",
    42: "Poste del techo",
    43: "Puerta delantera derecha",
    44: "Puerta delantera izquierda",
    45: "Puerta trasera derecha",
    46: "Puerta trasera izquierda",
    47: "Radiador",
    48: "Rejilla, parrilla",
    49: "Rueda",
    50: "Silenciador, el mofle",
    51: "Tapa de combustible",
    52: "Tapa de rueda",
    53: "Techo",
    54: "Techo corredizo",
    55: "Ventana delantera derecha",
    56: "Ventana delantera izquierda",
    57: "Ventana trasera derecha",
    58: "Ventana trasera izquierda",
    59: "Ventanilla delantera derecha",
    60: "Ventanilla delantera izquierda",
    61: "Ventanilla trasera derecha",
    62: "Ventanilla trasera izquierda",
    63: "Volante"
}

## Diccionario para Tipos de Daño (completo)
label_to_cls_danos = {
    1: "Abolladura",
    2: "Arañazo",
    3: "Corrosión",
    4: "Deformación",
    5: "Desprendimiento",
    6: "Fractura",
    7: "Rayón",
    8: "Rotura"
}

## Diccionario para Sugerencia (completo)
label_to_cls_sugerencia = {
    1: "Reparar",
    2: "Reemplazar"
}

# =============================================
# 1. CONFIGURACIÓN INICIAL
# =============================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42
torch.manual_seed(SEED)
batch_size = 32

# =============================================
# 2. DATASET Y TRANSFORMACIONES MEJORADAS
# =============================================
class VehiculoDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file, sep='|')
        self.root_dir = root_dir
        self.transform = transform
        self.class_distribution = self._compute_class_distribution()
        
    def _compute_class_distribution(self):
        return {
            'dano': Counter(self.data.iloc[:, 1]),
            'pieza': Counter(self.data.iloc[:, 2]),
            'sugerencia': Counter(self.data.iloc[:, 3])
        }
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        
        labels = {
            'dano': torch.tensor(self.data.iloc[idx, 1] - 1, dtype=torch.long),
            'pieza': torch.tensor(self.data.iloc[idx, 2] - 1, dtype=torch.long),
            'sugerencia': torch.tensor(self.data.iloc[idx, 3] - 1, dtype=torch.long)
        }
        
        if self.transform:
            image = self.transform(image)
            
        return image, labels

# Transformaciones mejoradas con aumentación más agresiva
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.4, 1.0)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
        transforms.RandomRotation(45),
        transforms.RandomPerspective(distortion_scale=0.3, p=0.5),
        transforms.RandomGrayscale(p=0.2),
        transforms.GaussianBlur(kernel_size=3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# =============================================
# 3. MODELO MEJORADO CON TRANSFER LEARNING EN FASES
# =============================================
class EnhancedMultiTaskModel(nn.Module):
    def __init__(self, num_danos, num_piezas, num_sugerencias):
        super().__init__()
        # Backbone principal con pesos preentrenados
        weights = EfficientNet_B0_Weights.IMAGENET1K_V1
        self.base_model = torchvision_models.efficientnet_b0(weights=weights)
        
        # Congelar todos los parámetros inicialmente
        for param in self.base_model.parameters():
            param.requires_grad = False
            
        in_features = self.base_model.classifier[1].in_features
        self.base_model.classifier = nn.Identity()
        
        # Capas compartidas
        self.shared_features = nn.Sequential(
            nn.Linear(in_features, 2048),
            nn.BatchNorm1d(2048),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.5)
        )
        
        # Cabezales mejorados
        # Cabeza para daños
        self.dano_head = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.LayerNorm(1024),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),
            nn.Linear(1024, num_danos)
        )
        
        # Cabeza para piezas (mayor capacidad)
        self.pieza_head = nn.Sequential(
            nn.Linear(2048, 2048),
            nn.LayerNorm(2048),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.4),
            nn.Linear(2048, 1024),
            nn.LeakyReLU(0.1),
            nn.Linear(1024, num_piezas)
        )
        
        # Cabeza para sugerencia
        self.sugerencia_head = nn.Sequential(
            nn.Linear(2048, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, num_sugerencias)
        )
        
        self._init_weights()
    
    def _init_weights(self):
        for head in [self.dano_head, self.pieza_head, self.sugerencia_head]:
            for m in head.modules():
                if isinstance(m, nn.Linear):
                    nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')
                    nn.init.constant_(m.bias, 0)
    
    def unfreeze_backbone(self, unfreeze_layers=5):
        """Descongela capas superiores del backbone"""
        total_layers = len(list(self.base_model.parameters()))
        for i, param in enumerate(self.base_model.parameters()):
            if i >= total_layers - unfreeze_layers:
                param.requires_grad = True
    
    def forward(self, x):
        base_features = self.base_model(x)
        shared = self.shared_features(base_features)
        
        return {
            'dano': self.dano_head(shared),
            'pieza': self.pieza_head(shared),
            'sugerencia': self.sugerencia_head(shared)
        }


# =============================================
# 4. FUNCIONES DE PÉRDIDA MEJORADAS
# =============================================
# Función para crear samplers balanceados
def get_sampler_weights(dataset, task):
    # Obtener la columna correcta según la tarea
    col_idx = 1 if task == 'dano' else 2 if task == 'pieza' else 3
    class_values = dataset.data.iloc[:, col_idx].values
    
    # Calcular conteos para las clases presentes
    unique_classes, counts = np.unique(class_values, return_counts=True)
    weights = 1. / counts
    
    # Crear un mapeo de clase a peso
    class_to_weight = {cls: weight for cls, weight in zip(unique_classes, weights)}
    
    # Asignar pesos a cada muestra
    samples_weights = np.array([class_to_weight[cls] for cls in class_values])
    
    return torch.from_numpy(samples_weights).double()


class BalancedFocalLoss(nn.Module):
    def __init__(self, alpha='auto', gamma=2.0, num_classes=None):
        super().__init__()
        self.gamma = gamma
        if alpha == 'auto' and num_classes:
            self.alpha = torch.ones(num_classes, device=device) / num_classes
        else:
            self.alpha = alpha
    
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        
        if self.alpha is not None:
            alpha = self.alpha[targets]
            loss = alpha * (1-pt)**self.gamma * ce_loss
        else:
            loss = (1-pt)**self.gamma * ce_loss
            
        return loss.mean()

def create_criterion(dataset, device):
    # Calcular pesos automáticamente basados en clases presentes
    def get_weights(task, num_classes):
        col_idx = 1 if task == 'dano' else 2 if task == 'pieza' else 3
        class_values = dataset.data.iloc[:, col_idx].values
        unique_classes, counts = np.unique(class_values, return_counts=True)
        weights = torch.zeros(num_classes, device=device)
        for cls, count in zip(unique_classes, counts):
            weights[cls-1] = 1.0 / count
        return weights / weights.sum()
    
    return {
        'dano': BalancedFocalLoss(alpha=get_weights('dano', 8), gamma=2),
        'pieza': BalancedFocalLoss(alpha=get_weights('pieza', 63), gamma=2),
        'sugerencia': BalancedFocalLoss(alpha=torch.tensor([0.4, 0.6], device=device))
    }

# =============================================
# 5. ENTRENAMIENTO POR ETAPAS (VERSIÓN CORREGIDA)
# =============================================
def train_phase(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, phase_name, is_reduce_lr=False):
    print(f"\n=== Fase: {phase_name} ===")
    best_metrics = {'dano': 0, 'pieza': 0, 'sugerencia': 0}
    best_val_loss = float('inf')
    patience = 3
    epochs_no_improve = 0
    
    for epoch in range(num_epochs):
        model.train()
        train_metrics = {task: 0 for task in best_metrics}
        total_loss = 0
        
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = {k: v.to(device) for k, v in labels.items()}
            
            optimizer.zero_grad()
            outputs = model(inputs)
            
            losses = {
                task: criterion[task](outputs[task], labels[task]) 
                for task in best_metrics
            }
            
            task_weights = {
                'dano': 0.5 * (1 - best_metrics['dano']),
                'pieza': 0.3 * (1 - best_metrics['pieza']),
                'sugerencia': 0.2 * (1 - best_metrics['sugerencia'])
            }
            batch_loss = sum(w * losses[task] for task, w in task_weights.items())
            
            batch_loss.backward()
            optimizer.step()
            
            total_loss += batch_loss.item()
            for task in best_metrics:
                _, preds = torch.max(outputs[task], 1)
                train_metrics[task] += (preds == labels[task]).sum().item()
        
        # Validación
        model.eval()
        val_metrics = {task: 0 for task in best_metrics}
        val_loss = 0
        total_samples = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = {k: v.to(device) for k, v in labels.items()}
                outputs = model(inputs)
                
                batch_size = inputs.size(0)
                total_samples += batch_size
                
                losses = {
                    task: criterion[task](outputs[task], labels[task]) 
                    for task in best_metrics
                }
                val_loss += sum(losses.values()).item()
                
                for task in best_metrics:
                    _, preds = torch.max(outputs[task], 1)
                    val_metrics[task] += (preds == labels[task]).sum().item()
        
        # Early stopping
        val_loss /= len(val_loader)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
        
        # Actualizar scheduler
        if is_reduce_lr:
            scheduler.step(0.5*val_metrics['dano'] + 0.3*val_metrics['pieza'] + 0.2*val_metrics['sugerencia'])
        else:
            scheduler.step()
        
        # Resultados
        print(f"\nEpoch {epoch+1}/{num_epochs} - Loss: {total_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}")
        for task in best_metrics:
            val_acc = val_metrics[task] / total_samples
            train_acc = train_metrics[task] / len(train_loader.dataset)
            print(f"{task.capitalize()} - Train: {train_acc:.4f}, Val: {val_acc:.4f}")
            
            if val_acc > best_metrics[task]:
                best_metrics[task] = val_acc
                torch.save(model.state_dict(), f'best_{task}_model.pth')
    
    return best_metrics

# =============================================
# 6. PIPELINE COMPLETO (VERSIÓN CORREGIDA)
# =============================================
# Last Execution 10:04:03 PM
# Execution Time 63m 47.5s
# Overhead Time 42m 25.7s
# Render Times
# VS Code Builtin Notebook Output Renderer 3ms

def main():
    # Cargar datos
    image_dir = 'data/fotos_siniestros/'
    train_dataset = VehiculoDataset(
        csv_file='data/fotos_siniestros/datasets/train.csv',
        root_dir=image_dir,
        transform=data_transforms['train']
    )
    val_dataset = VehiculoDataset(
        csv_file='data/fotos_siniestros/datasets/val.csv',
        root_dir=image_dir,
        transform=data_transforms['val']
    )
    
    # Crear samplers balanceados para cada tarea
    sampler_dano = WeightedRandomSampler(
        get_sampler_weights(train_dataset, 'dano'), 
        len(train_dataset)
    )
    sampler_pieza = WeightedRandomSampler(
        get_sampler_weights(train_dataset, 'pieza'), 
        len(train_dataset)
    )
    
    # Usar el sampler para la tarea más desbalanceada (pieza)
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        sampler=sampler_pieza,
        num_workers=4,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    
    # Crear modelo
    model = EnhancedMultiTaskModel(8, 63, 2).to(device)
    criterion = create_criterion(train_dataset, device)
    
    # Fase 1: Solo cabezales (10 épocas)
    optimizer = optim.AdamW([
        {'params': model.shared_features.parameters()},
        {'params': model.dano_head.parameters()},
        {'params': model.pieza_head.parameters()},
        {'params': model.sugerencia_head.parameters()}
    ], lr=0.001, weight_decay=1e-3)
    
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer, 
        max_lr=0.01, 
        steps_per_epoch=len(train_loader), 
        epochs=10
    )
    
    train_phase(model, train_loader, val_loader, criterion, optimizer, scheduler, 10, "Entrenamiento inicial (cabezales)")
    
    # Fase 2: Descongelar capas superiores (15 épocas)
    model.unfreeze_backbone(unfreeze_layers=10)
    
    optimizer = optim.AdamW([
        {'params': model.base_model.parameters(), 'lr': 0.0001},
        {'params': model.shared_features.parameters(), 'lr': 0.0005},
        {'params': model.dano_head.parameters(), 'lr': 0.001},
        {'params': model.pieza_head.parameters(), 'lr': 0.001},
        {'params': model.sugerencia_head.parameters(), 'lr': 0.005}
    ], weight_decay=1e-3)
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='max', 
        factor=0.5, 
        patience=2, 
        verbose=True
    )# =============================================
# 1. CONFIGURACIÓN INICIAL (MEJORADA)
# =============================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42
torch.manual_seed(SEED)
batch_size = 32
min_samples_per_class = 15  # Mínimo de muestras por clase
    # Fase 3: Fine-tuning completo (15 épocas)
    for param in model.base_model.parameters():
        param.requires_grad = True
    
    train_phase(model, train_loader, val_loader, criterion, optimizer, scheduler, 15, "Fine-tuning completo", is_reduce_lr=True)

if __name__ == "__main__":
    main()

# =============================================
# EVALUACIÓN FINAL Y MATRICES DE CONFUSIÓN
# =============================================
def plot_confusion_matrix(y_true, y_pred, classes, title='Matriz de Confusión', cmap=plt.cm.Blues, figsize=(12, 10)):
    """Función mejorada para visualizar matrices de confusión"""
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=figsize)
    sns.heatmap(cm, annot=True, fmt='d', cmap=cmap, 
                xticklabels=classes, yticklabels=classes,
                cbar=False, linewidths=0.5, linecolor='gray')
    
    plt.title(title, fontsize=14, pad=20)
    plt.xlabel('Predicción', fontsize=12)
    plt.ylabel('Real', fontsize=12)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.yticks(rotation=0, fontsize=10)
    plt.tight_layout()
    
    # Guardar la figura
    filename = title.lower().replace(' ', '_') + '.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

def evaluate_task(model, loader, task_name, label_dict):
    """Evalúa un modelo en una tarea específica y genera métricas"""
    model.eval()
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            
            _, preds = torch.max(outputs[task_name], 1)
            y_true.extend(labels[task_name].cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    
    # Convertir a arrays numpy
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Filtrar solo clases presentes
    present_labels = np.unique(np.concatenate([y_true, y_pred]))
    class_names = [label_dict[label+1] for label in present_labels]
    
    # Manejo especial para piezas (muchas clases)
    if task_name == 'pieza' and len(present_labels) > 20:
        label_counts = {label: np.sum(y_true == label) for label in present_labels}
        top_labels = sorted(present_labels, key=lambda x: label_counts[x], reverse=True)[:20]
        mask = np.isin(y_true, top_labels)
        y_true_filtered = y_true[mask]
        y_pred_filtered = y_pred[mask]
        class_names = [label_dict[label+1] for label in top_labels]
    else:
        y_true_filtered = y_true
        y_pred_filtered = y_pred
    
    print(f"\n=== Resultados para {task_name.capitalize()} ===")
    
    # Obtener las etiquetas presentes en los datos
    labels_present = np.unique(np.concatenate([y_true, y_pred]))
    target_names = [label_dict[label+1] for label in labels_present]
    
    print(classification_report(
        y_true, 
        y_pred, 
        labels=labels_present,
        target_names=target_names,
        zero_division=0,
        digits=4
    ))
    
    plot_confusion_matrix(
        y_true_filtered, 
        y_pred_filtered, 
        classes=class_names,
        title=f'Matriz de Confusión - {task_name.capitalize()}',
        figsize=(15, 12) if task_name == 'pieza' else (10, 8)
    )

# Diccionarios de etiquetas
label_dicts = {
    'dano': label_to_cls_danos,
    'pieza': label_to_cls_piezas,
    'sugerencia': label_to_cls_sugerencia
}

# =============================================
# CARGA Y EVALUACIÓN DE MODELOS ENTRENADOS
# =============================================
def load_and_evaluate_models():
    """Carga los modelos entrenados y genera las matrices de confusión"""
    # Cargar los mejores modelos para cada tarea
    tasks = ['dano', 'pieza', 'sugerencia']
    model_instances = {}  # Cambio de nombre para evitar conflicto
    
    for task in tasks:
        model_path = f'best_{task}_model.pth'
        if os.path.exists(model_path):
            # Crear nueva instancia del modelo
            model = EnhancedMultiTaskModel(8, 63, 2).to(device)
            model.load_state_dict(torch.load(model_path))
            model_instances[task] = model
            print(f"Modelo para {task} cargado exitosamente")
        else:
            print(f"Advertencia: No se encontró el modelo para {task} en {model_path}")
    
    # Verificar si val_loader está definido
    if 'val_loader' not in locals():
        val_dataset = VehiculoDataset(
            csv_file='data/fotos_siniestros/datasets/val.csv',
            root_dir='data/fotos_siniestros/',
            transform=data_transforms['val']
        )
        val_loader = DataLoader(
            val_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=4,
            pin_memory=True
        )
    
    # Evaluar cada tarea
    for task in tasks:
        if task in model_instances:
            print(f"\n{'='*50}")
            print(f"Evaluando modelo para tarea: {task.upper()}")
            print(f"{'='*50}")
            evaluate_task(model_instances[task], val_loader, task, label_dicts[task])

# Ejecutar la evaluación
if __name__ == "__main__":
    load_and_evaluate_models()

==================================================
## Salida
==================================================

=== Fase: Entrenamiento inicial (cabezales) ===

Epoch 1/10 - Loss: 0.8768, Val Loss: 2.2638
Dano - Train: 0.2232, Val: 0.1760
Pieza - Train: 0.1301, Val: 0.0146
Sugerencia - Train: 0.5590, Val: 0.5561

Epoch 2/10 - Loss: 0.2723, Val Loss: 1.8339
Dano - Train: 0.2518, Val: 0.1839
Pieza - Train: 0.2279, Val: 0.0056
Sugerencia - Train: 0.5701, Val: 0.5830

Epoch 3/10 - Loss: 0.2153, Val Loss: 1.6102
Dano - Train: 0.2846, Val: 0.2164
Pieza - Train: 0.2523, Val: 0.0101
Sugerencia - Train: 0.5747, Val: 0.5426

Epoch 4/10 - Loss: 0.1784, Val Loss: 1.1537
Dano - Train: 0.2966, Val: 0.2052
Pieza - Train: 0.2583, Val: 0.0101
Sugerencia - Train: 0.5909, Val: 0.5303

Epoch 5/10 - Loss: 0.1600, Val Loss: 1.1449
Dano - Train: 0.3007, Val: 0.2646
...
Sugerencia - Train: 0.6042, Val: 0.5235
Early stopping at epoch 10

=== Fase: Fine-tuning parcial ===
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...
/data/Python/VehiculosVerificationDeDannosEtiquetas/.venv/lib64/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

Epoch 1/15 - Loss: 0.6888, Val Loss: 0.9349
Dano - Train: 0.3270, Val: 0.2096
Pieza - Train: 0.2721, Val: 0.0022
Sugerencia - Train: 0.5577, Val: 0.5235

Epoch 2/15 - Loss: 0.0822, Val Loss: 0.2629
Dano - Train: 0.3021, Val: 0.2646
Pieza - Train: 0.2823, Val: 0.0090
Sugerencia - Train: 0.5526, Val: 0.5235

Epoch 3/15 - Loss: 0.0482, Val Loss: 0.2140
Dano - Train: 0.3247, Val: 0.2993
Pieza - Train: 0.3003, Val: 0.0247
Sugerencia - Train: 0.5756, Val: 0.5639

Epoch 4/15 - Loss: 0.0413, Val Loss: 0.2403
Dano - Train: 0.3141, Val: 0.3128
Pieza - Train: 0.3367, Val: 0.0067
Sugerencia - Train: 0.5812, Val: 0.4843

Epoch 5/15 - Loss: 0.0335, Val Loss: 0.3034
Dano - Train: 0.3455, Val: 0.2534
Pieza - Train: 0.3372, Val: 0.0112
Sugerencia - Train: 0.5530, Val: 0.6132
...
Dano - Train: 0.3589, Val: 0.2960
Pieza - Train: 0.4327, Val: 0.0348
Sugerencia - Train: 0.6135, Val: 0.5774
Early stopping at epoch 9
)

Modelo para dano cargado exitosamente
Modelo para pieza cargado exitosamente
Modelo para sugerencia cargado exitosamente

==================================================
Evaluando modelo para tarea: DANO
==================================================

=== Resultados para Dano ===

                    precision    recall  f1-score   support

        Abolladura     0.3395    0.4015    0.3679       274
            Arañazo     0.3107    0.2286    0.2634       140
        Deformación     0.2000    0.0408    0.0678        49
    Desprendimiento     0.1746    0.2178    0.1938       101
          Fractura     0.0460    0.1212    0.0667        33
              Rayón     0.1000    0.1333    0.1143        15
            Rotura     0.4910    0.3893    0.4343       280

          accuracy                         0.3150       892
          macro avg     0.2374    0.2189    0.2154       892
      weighted avg     0.3413    0.3150    0.3207       892

==================================================
Evaluando modelo para tarea: PIEZA
==================================================

=== Resultados para Pieza ===

                                        precision    recall  f1-score   support

          Antiniebla delantero derecho     0.0000    0.0000    0.0000         0
        Antiniebla delantero izquierdo     0.0000    0.0000    0.0000         6
                      Brazo del techo     0.0000    0.0000    0.0000         4
                                  Capó     0.0000    0.0000    0.0000        55
                Espejo lateral derecho     0.0000    0.0000    0.0000         6
              Espejo lateral izquierdo     0.0000    0.0000    0.0000         6
                        Faros derecho     0.0000    0.0000    0.0000        36
                      Faros izquierdo     0.0000    0.0000    0.0000        50
        Guardabarros delantero derecho     0.0000    0.0000    0.0000        58
      Guardabarros delantero izquierdo     0.0000    0.0000    0.0000        77
          Guardabarros trasero derecho     0.0000    0.0000    0.0000        19
        Guardabarros trasero izquierdo     0.0588    0.0357    0.0444        28
                      Limpiaparabrisas     0.0000    0.0000    0.0000         0
      Luz indicadora delantera derecha     0.0769    1.0000    0.1429         6
    Luz indicadora delantera izquierda     0.0000    0.0000    0.0000         8
      Luz indicadora trasera izquierda     0.0000    0.0000    0.0000         0
                  Luz trasera derecho     0.0000    0.0000    0.0000        20
    ...
                              accuracy                         0.0504       892
                            macro avg     0.0340    0.1039    0.0402       892
                          weighted avg     0.0205    0.0504    0.0234       892

==================================================
Evaluando modelo para tarea: SUGERENCIA
==================================================

=== Resultados para Sugerencia ===

                  precision    recall  f1-score   support

        Reparar     0.6245    0.9006    0.7376       543
      Reemplazar     0.5046    0.1576    0.2402       349

        accuracy                         0.6099       892
      macro avg     0.5646    0.5291    0.4889       892
    weighted avg     0.5776    0.6099    0.5430       892

### Areas for Improvement
### Model Architecture:

1. Consider experimenting with different architectures or fine-tuning strategies.
Evaluate the effectiveness of the current layers and dropout rates.
Data Augmentation:

2. Review the data augmentation techniques used and consider adding or modifying them to improve model robustness.
Training Process:

3. Analyze the learning rate scheduling and optimizer settings.
Implement early stopping based on validation metrics to prevent overfitting.
Evaluation Metrics:

4. Improve the evaluation metrics to provide more insights into model performance, especially for imbalanced classes.
Logging and Visualization:

5. Enhance logging during training to capture more detailed metrics.
Improve visualization of training and validation losses.
Plan for Improvements
Model Architecture:

6. Experiment with additional layers or different activation functions.
Adjust dropout rates based on validation performance.
Data Augmentation:

7. Add more aggressive augmentations or consider using techniques like CutMix or MixUp.
Training Process:

8. Implement a more sophisticated learning rate scheduler.
Add early stopping based on validation loss.
Evaluation Metrics:

9. Include additional metrics such as F1 score, precision, and recall for each class.
Visualize confusion matrices for better insights.
Logging and Visualization:

10. Use TensorBoard or similar tools for better visualization of training metrics.

### Steps to Implement Improvements
1. Model Architecture:
    - Experiment with additional layers or different activation functions.
    - Adjust dropout rates based on validation performance.

2. Data Augmentation:
    - Add more aggressive augmentations or consider using techniques like CutMix or MixUp.

3. Training Process:
    - Implement a more sophisticated learning rate scheduler.
    - Add early stopping based on validation loss.

4. Evaluation Metrics:
    - Include additional metrics such as F1 score, precision, and recall for each class.
    - Visualize confusion matrices for better insights.

5. Logging and Visualization:
    - Use TensorBoard or similar tools for better visualization of training metrics.

### Key Observations:
1. Model Architecture: The current model uses a ResNet50 backbone with separate heads for damage type, vehicle part, and suggestion.
2. Data Augmentation: The notebook includes data augmentation techniques, but there may be room for improvement.
3. Training Process: The training process is defined, but enhancements can be made in learning rate scheduling and early stopping.
4. Evaluation Metrics: The evaluation metrics are currently limited; additional metrics like F1 score, precision, and recall could provide better insights.
5. Logging and Visualization: There is a need for enhanced logging and visualization of training metrics.